# Colab-specific instructions start here

In [1]:
# Here we use a Conda environment inside Google Colab. Blocks specific for Colab
# (like this one) mention "condacolab". On "normal" platforms the procedure
# for installation may be different - you need to check the system's documentation.

# Colab notebooks are "brittle": in the course of time Colab is updated
# and dependencies no longer work properly. Proper HPC platforms are more
# stable (and supported)

# After executing this cell, Colab restarts.

!pip install -q condacolab
import condacolab
condacolab.install_miniforge()

✨🍰✨ Everything looks OK!


In [2]:
# Verify Python version
import sys
sys.version

'3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]'

In [3]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [4]:
# Colab-specific workaround for a weird error upon shell escape:
# NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# Generic installation instructions

In [5]:
# Verify GPU availability and type. If you get an error, check that
# "Runtime / Runtime type / GPU" is selected.
!nvidia-smi

Sun Mar 30 08:23:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
# Install OpenMM. Takes a long time.
!conda install -q -c conda-forge openmm pdbfixer

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - openmm
    - pdbfixer


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.1.31  |       hbcca054_0         154 KB  conda-forge
    certifi-2025.1.31          |     pyhd8ed1ab_0         159 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    cudatoolkit-11.8.0         |      h4ba93d1_13       682.5 MB  conda-forge
    kernel-headers_linux-64-3.10.0|      he073ed8_18         921 KB  conda-forge
    libblas-3.9.0              |31_h59b9bed_openblas          16 KB  conda-forge
    libcblas-3.9.0             |31_he106b2a_openblas          16 KB  conda-forge
    libgfortran-14.2.0         |       h69a702a_2          52 KB  conda-forge
    libgfortran5-14.2.0    

# Tests

In [7]:
# A quick test
import openmm.testInstallation
openmm.testInstallation.main()


OpenMM Version: 8.2
Git Revision: 53770948682c40bd460b39830d4e0f0fd3a4b868

There are 3 Platforms available:

1 Reference - Successfully computed forces
2 CPU - Successfully computed forces
3 CUDA - Successfully computed forces

Median difference in forces between platforms:

Reference vs. CPU: 6.29552e-06
Reference vs. CUDA: 6.75161e-06
CPU vs. CUDA: 7.51936e-07

All differences are within tolerance.


In [8]:
# Note the "ns_per_day" figure when using the GPU
!(cd /usr/local/share/openmm/examples; python benchmark.py --platform CUDA --test pme --seconds 5  --precision mixed)

hostname: 4df7bb3a6770
timestamp: 2025-03-30T08:25:56.189772+00:00
openmm_version: 8.2.0.dev-5377094
cpuinfo: Intel(R) Xeon(R) CPU @ 2.00GHz
cpuarch: x86_64
system: Linux
nvidia_driver: 550.54.15
gpu: Tesla T4
test: pme
constraints: HBonds
hydrogen_mass: 1.5
cutoff: 0.9
ensemble: NVT
precision: mixed
timestep_in_fs: 4.0
platform: CUDA
platform_properties: {'DeviceIndex': '0', 'DeviceName': 'Tesla T4', 'UseBlockingSync': 'false', 'Precision': 'mixed', 'UseCpuPme': 'false', 'CudaCompiler': '', 'TempDirectory': '/tmp', 'CudaHostCompiler': '', 'DisablePmeStream': 'false', 'DeterministicForces': 'false'}
steps: 6242
elapsed_time: 4.178066
ns_per_day: 516.3238685075821



In [14]:
# CUDA (NVIDIA GPU) is the fastest platform. You can the others with... And check how the "ns_per_day" varies.

# !(cd /usr/local/share/openmm/examples; python benchmark.py --platform CUDA --test pme --seconds 5  --precision mixed)
# !(cd /usr/local/share/openmm/examples; python benchmark.py --platform Reference --test pme --seconds 5  --precision mixed)
# !(cd /usr/local/share/openmm/examples; python benchmark.py --platform CPU --test pme --seconds 5  --precision mixed)



---


# Here begins the proper simulations tutorial

In [15]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from pdbfixer import *
from sys import stdout

## Download, fix missing atoms, solvate

Can also be done on the command line with the `pdbfixer` executable.

For further info on PDBFixer, refer to the manual: \
https://htmlpreview.github.io/?https://github.com/openmm/pdbfixer/blob/master/Manual.html

In [26]:
# Retrieve the structure from the RCSB
fixer = PDBFixer(pdbid="6H1F")

# Add missing (unresolved) residues. We don't want to model anything.
fixer.findMissingResidues()
fixer.missingResidues = {}

# Add missing (unresolved) atoms
fixer.findMissingAtoms()
fixer.addMissingAtoms()

# Protonate (roughly) at chosen pH
fixer.addMissingHydrogens(pH=7.0)

# Explicit solvent: 10 nm^3 box
fixer.addSolvent(boxSize=10 * Vec3(1, 1, 1))

# Save the file so it can be inspected
PDBFile.writeFile(fixer.topology, fixer.positions, open("6H1F-fixed.pdb", "w"))


## Modeling

Modelling step, sometimes unnecessary.
Here it is needed to remove an "SCN" (THIOCYANATE ION) residue.

We're going to create a Modeller object (http://docs.openmm.org/latest/api-python/generated/openmm.app.modeller.Modeller.html?highlight=modeller).



If you want to access the link information directly from the code, type:\
`help(Modeller)`


In [27]:
# There is an "SCN" residue to remove
modeller = Modeller(fixer.topology, fixer.positions)

res_SCN = [r for r in modeller.topology.residues() if r.name == "SCN"]
modeller.delete(res_SCN)

PDBFile.writeFile(
    modeller.topology, modeller.positions, open("6H1F-modelled.pdb", "w"), keepIds=True
)



---


## Create integration-related objects

The OpenMM objects we have to create at this point are:
- ForceField (http://docs.openmm.org/latest/api-python/generated/openmm.app.forcefield.ForceField.html?highlight=forcefield)
- System (http://docs.openmm.org/latest/api-python/generated/openmm.openmm.System.html?highlight=system)
- Simulation (http://docs.openmm.org/latest/api-python/generated/openmm.app.simulation.Simulation.html?highlight=simulation)


If you want to access this information directly from the code, type:\
`help(<object name>)`

In [28]:
# The FF object holds the parameters
forcefield = ForceField("amber14-all.xml", "amber14/tip3pfb.xml")

In [29]:
# This specifies the system to be simulated.
system = forcefield.createSystem(
    modeller.topology,
    nonbondedMethod=PME,
    nonbondedCutoff=1 * nanometer,
    constraints=HBonds,
)

In [30]:
# Specify the integrator: temperature, relaxation time, timestep (important)
integrator = LangevinMiddleIntegrator(300 * kelvin, 1 / picosecond, 0.004 * picoseconds)

In [31]:
# The barostat is added to the system so that density is controlled
# in addition to temperature.

# Pressure, Temperature (only used for calculation),
# Frequency (how frequently the system should update the box size)
barostat = MonteCarloBarostat(1.0 * atmosphere, 300.0 * kelvin, 25)

system.addForce(barostat)


5

In [32]:
# Combines the molecular topology, system, and integrator to begin a new simulation.

simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)



---


## Minimize energy

In [33]:
# Perform local energy minimization
print("Minimizing energy...")
simulation.minimizeEnergy(maxIterations=500)
print("Minimization done.")

# Write the minimized coordinates (for checking)
PDBFile.writeFile(
    simulation.topology,
    simulation.context.getState(getPositions=True).getPositions(),
    open("6H1F-minimized.pdb", "w"),
    keepIds=True,
)

Minimizing energy...
Minimization done.




---


## Integrate

In [34]:
Nsteps = 5000

In [35]:
# When the simulation runs, it will write the trajectory to a file called "output.dcd"
simulation.reporters.append(
    DCDReporter("output.dcd", reportInterval=1000, enforcePeriodicBox=False)
)

In [36]:
# Also report infomation to the screen as the simulation runs

simulation.reporters.append(
    StateDataReporter(
        stdout,
        100,
        step=True,
        time=True,
        potentialEnergy=True,
        kineticEnergy=True,
        totalEnergy=True,
        temperature=True,
        volume=True,
        density=True,
        progress=True,
        remainingTime=True,
        speed=True,
        elapsedTime=True,
        separator=" ",
        totalSteps=Nsteps,
    )
)

In [37]:
# Finally run the simulation for the given timesteps
print("Running simulation...")
simulation.step(Nsteps)

Running simulation...
#"Progress (%)" "Step" "Time (ps)" "Potential Energy (kJ/mole)" "Kinetic Energy (kJ/mole)" "Total Energy (kJ/mole)" "Temperature (K)" "Box Volume (nm^3)" "Density (g/mL)" "Speed (ns/day)" "Elapsed Time (s)" "Time Remaining"
2.0% 100 0.4000000000000003 -1738110.0206970386 97740.9262883632 -1640369.0944086753 119.78694466673052 989.3554834043607 0.987443514029133 0 0.0016360282897949219 --
4.0% 200 0.8000000000000006 -1693762.7825275995 149562.60887418396 -1544200.1736534154 183.2973006677626 986.3967505050822 0.9904053867336419 110 0.31374454498291016 0:15
6.0% 300 1.2000000000000008 -1662080.5676838495 181340.52379066116 -1480740.0438931882 222.2429039096804 986.3967505050822 0.9904053867336419 110 0.6280112266540527 0:14
8.0% 400 1.6000000000000012 -1639187.942180668 201890.89013931967 -1437297.0520413483 247.42852154364053 985.6772791314869 0.9911283092754268 107 0.9667184352874756 0:14
10.0% 500 2.0000000000000013 -1622228.438274418 214780.6480020187 -1407447.7



---


# Results

The simulation is completed. Now download the minimized PDB file (that gives the starting coordinates and the identity of the atoms) and the DCD file (a binary file, providing the trajectory, i.e. a series of snapshots of the coordinates). They can be best visualized locally on e.g. PyMol or VMD.


# Visualize

Here we use a cloud-based viewer, but you should better download the results and use VMD or PyMOL.

In [38]:
#!conda install -q MDAnalysis mdtraj
!pip install py3Dmol
import py3Dmol

In [39]:
view = py3Dmol.view()
view.addModel(open('6H1F-minimized.pdb', 'r').read(),'pdb')
view.addStyle({"cartoon": {'color': 'spectrum'}})
view.zoomTo()


Output hidden; open in https://colab.research.google.com to view.